In [35]:
import numpy as np 
import scipy.stats as scistats

In [68]:
def fatigue_strength_fraction(Sut):

    if Sut < 490:
        f = 0.9
    else:
        Se = 0.5 * Sut
        Ne = 1E6
        sigmaf = Sut + 345
        b = - np.log10(sigmaf/Se)/np.log10( 2 * Ne)
        f = sigmaf / Sut * (2 * 10**3) ** b
    return f

def number_cycles_to_failure(sigma_reversed,Sut,Se):
    f = fatigue_strength_fraction(Sut)
    a = (f * Sut)**2 / Se 
    b = (-1/3 * np.log10(f * Sut/Se))
    N = (sigma_reversed/a)**(1/b)
    return N


def marin_surface_factor(Sut,surface_finish): 
    '''
    Sut: Ultimate strength in MPa
    
    surface_finish must be one of the following options: 
    * ground
    * machined
    * cold-drawn
    * hot-rolled
    * as-forged    
    '''
    if surface_finish == "ground": 
        
        a = 1.58
        b = -0.085
    
    elif (surface_finish == "machined") or (surface_finish == "cold-drawn"):
        a = 4.51
        b = -0.265
            
    elif (surface_finish == "hot-rolled"): 
        
        a = 57.7
        b = -0.718
        
    elif (surface_finish == "as-forged"): 
        
        a = 272.
        b = -0.995
        
    else:
        raise ValueError("You supplied the following surface finish: '{}' which does not exist. Please check your spelling - only the following options are available:  \n* ground\n* machined\n* cold-drawn\n* hot-rolled\n* as-forged   ".format(surface_finish))
        
    return a * Sut ** b

def marin_loading_factor(load_type): 
    
    if load_type == "bending": 
        return 1 
    if load_type == "axial": 
        return 0.85
    if load_type == "torsion":
        return 0.59
    raise ValueError("You must supply bending, axial or torsion as an input.")
def marin_temperature_factor(temperature_degrees_celcius): 
    x = 20,50,100,150,200,250,300,350,400,450,500,550,660
    y = 1,1.010,1.020,1.025,1.020,1.00,0.975,0.943,0.9,0.843,0.768,0.672,0.549
    
    import scipy.interpolate as sciint
    
    kd = float(sciint.interp1d(x,y,kind="linear")(temperature_degrees_celcius))

    #kd = 0.975 + 0.432 * 1E-3 * Tf - 0.115 * 1E-5 * Tf **2 + 0.104 * 1E-8 * Tf **3 - 0.595 * 1E-12 * Tf**4
    
    return kd
    
def marin_reliability_factor(reliability_fraction): 
    if (reliability_fraction < 0) | (reliability_fraction > 1):
        raise ValueError("Please insert a value between 0 and 1")
    return 1 - 0.08 * scistats.norm.ppf(reliability_fraction)


In [72]:

Sut = 600
ka = marin_surface_factor(Sut,"cold-drawn")
# kb = 
kc = marin_loading_factor("axial")
kd = marin_temperature_factor(20)
ke = marin_reliability_factor(0.99)

print("ka",ka)
print("kc",kc)
print("kd",kd)
print("ke",ke)


ka 0.8278782262842558
kc 0.85
kd 1.0
ke 0.8138921700767328


In [69]:
number_cycles_to_failure(205.71428571428572,690,190.40854046579244)

620051.2627241726